# Import

In [ ]:
!pip3 install pickle5
import pickle5 as pickle
import numpy as np
import cv2
import os
import pandas as pd
import tensorflow as tf
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics
from sklearn.utils import shuffle
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
import random
from keras.utils import np_utils

## CNN Parameters

In [ ]:
IMG_SIZE = 800
batch_size = 64
nb_classes =2
nb_epochs = 40
img_rows, img_columns = 12, 100
nb_filters = 32
nb_pool = 2
nb_conv = 3
learning_rate = 0.0001

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Normalization(),
    tf.keras.layers.Conv2D(32, (2,2), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.01),
                           input_shape=(100, 12,1)),
    #tf.keras.layers.Conv2D(64, (2,2), padding='same', activation=tf.keras.layers.LeakyReLU(alpha=0.01)),
    tf.keras.layers.MaxPooling2D((2, 1), strides=2),
    tf.keras.layers.Normalization(),
    #tf.keras.layers.MaxPooling2D((2, 1), strides=2),
    tf.keras.layers.Conv2D(16, (5,3), padding='same', activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 1), strides=3),
    tf.keras.layers.Normalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.MaxPooling2D((2, 2), strides=3),
    tf.keras.layers.Normalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),

    tf.keras.layers.Dense(2,  activation=tf.nn.softmax)
])

In [ ]:
import gc
gc.collect()

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])

## Load Data from Pickles And Train

In [ ]:
files = os.listdir("../../data/csv/pickles")
for i in range(0,len(files),2):

    with open("../../data/csv/pickles" + files[i], 'rb') as pickleFile:
        test = pickle.load(pickleFile)
    with open("../../data/csv/pickles" + files[i+1], 'rb') as pickleFile:
        test_1 = pickle.load(pickleFile)
    training = []
    for k in range(len(test)):
        new_array = cv2.resize(test[i][0], (IMG_SIZE, IMG_SIZE))
        training.append([new_array, test[k][1]])
    for k in range(len(test_1)):
        new_array = cv2.resize(test_1[i][0], (IMG_SIZE, IMG_SIZE))
        training.append([new_array, test_1[k][1]])
    X = []
    y = []
    random.shuffle(training)
    for features, label in training:
        X.append(features)
        y.append(label)

    X_reshaped = np.array(X).reshape(-1, 100, 12,1)
    category = np_utils.to_categorical(y, 2)
    X_train, X_test, y_train, y_test = train_test_split(X_reshaped, category, test_size = 0.1, random_state = 42,stratify=category)


    model.fit(X_train, y_train, batch_size = batch_size, epochs = nb_epochs, verbose = 1, validation_data = (X_test, y_test))
    print ( 10 * "-" , "file = ", i , 10*"-")
    import sklearn.metrics as metrics

    y_pred_ohe = model.predict(X_test)  # shape=(n_samples, 12)
    y_pred_labels = np.argmax(y_pred_ohe, axis=1)

    # shape=(n_samples, 12)
    y_test_true =  np.argmax(y_test, axis=1) # only necessary if output has one-hot-encoding, shape=(n_samples)

    confusion_matrix = metrics.confusion_matrix(y_true=y_test_true, y_pred=y_pred_labels)
    print(confusion_matrix)
    print ( 15 * "-" , "file = ", i , 15*"-")

    gc.collect()